In [13]:
from nnAudio import Spectrogram
from scipy.io import wavfile
import torch
import soundfile as sf
import numpy as np

In [19]:
sr, song = wavfile.read('./BAC009S0764W0124.wav') # Loading your audio
print(sr)
print(song)
print(song.shape)
print(song.dtype)
x = song
x = torch.tensor(x).float() # casting the array into a PyTorch Tensor

spec_layer = Spectrogram.STFT(n_fft=2048, freq_bins=None, hop_length=512,
                              window='hann', freq_scale='linear', center=True, pad_mode='reflect',
                              fmin=50,fmax=8000, sr=sr) # Initializing the model

spec = spec_layer(x) # Feed-forward your waveform to get the spectrogram
print(spec)

16000
[43 75 69 ...  7  6  3]
(83792,)
int16
sampling rate = 16000. Please make sure the sampling rate is correct in order toget a valid freq range
STFT kernels created, time used = 0.2142 seconds
tensor([[[[-4.0940e+03,  1.2600e+04],
          [ 8.5108e+03, -5.4930e+03],
          [-3.3631e+03, -1.7904e+03],
          ...,
          [ 8.2279e+03, -9.3340e+03],
          [-3.1990e+03,  2.0969e+03],
          [-1.2669e+03,  4.4488e+03]],

         [[ 3.4886e+03, -9.9620e+03],
          [-4.5364e+03,  4.1907e+02],
          [ 2.5074e+03,  7.1339e+03],
          ...,
          [-5.4819e+03,  3.9258e+01],
          [ 4.7221e+03,  6.5887e+01],
          [ 9.6492e+02, -3.4386e+03]],

         [[-3.4947e+03,  9.2981e+03],
          [-7.5164e+03,  8.1856e+02],
          [-5.3766e+03, -9.0889e+03],
          ...,
          [ 1.4317e+03,  5.7447e+03],
          [-3.1178e+03,  3.0740e+03],
          [-3.4351e+03,  5.6900e+02]],

         ...,

         [[ 6.7112e+01, -4.5737e+00],
          [-9.6

In [22]:
wav, sr = sf.read('./BAC009S0764W0124.wav', dtype='int16')
print(sr)
print(wav)
print(wav.shape)
print(wav.dtype)
print(np.allclose(wav, song))

x = wav
x = torch.tensor(x).float() # casting the array into a PyTorch Tensor

spec_layer = Spectrogram.STFT(n_fft=2048, freq_bins=None, hop_length=512,
                              window='hann', freq_scale='linear', center=True, pad_mode='reflect',
                              fmin=50,fmax=8000, sr=sr) # Initializing the model

wav_spec = spec_layer(x) # Feed-forward your waveform to get the spectrogram
print(wav_spec.shape)
print(wav_spec)
print(np.allclose(wav_spec, spec))

16000
[43 75 69 ...  7  6  3]
(83792,)
int16
True
sampling rate = 16000. Please make sure the sampling rate is correct in order toget a valid freq range
STFT kernels created, time used = 0.2495 seconds
torch.Size([1, 1025, 164, 2])
tensor([[[[-4.0940e+03,  1.2600e+04],
          [ 8.5108e+03, -5.4930e+03],
          [-3.3631e+03, -1.7904e+03],
          ...,
          [ 8.2279e+03, -9.3340e+03],
          [-3.1990e+03,  2.0969e+03],
          [-1.2669e+03,  4.4488e+03]],

         [[ 3.4886e+03, -9.9620e+03],
          [-4.5364e+03,  4.1907e+02],
          [ 2.5074e+03,  7.1339e+03],
          ...,
          [-5.4819e+03,  3.9258e+01],
          [ 4.7221e+03,  6.5887e+01],
          [ 9.6492e+02, -3.4386e+03]],

         [[-3.4947e+03,  9.2981e+03],
          [-7.5164e+03,  8.1856e+02],
          [-5.3766e+03, -9.0889e+03],
          ...,
          [ 1.4317e+03,  5.7447e+03],
          [-3.1178e+03,  3.0740e+03],
          [-3.4351e+03,  5.6900e+02]],

         ...,

         [[ 6.7112